This tutorial uses openAIs GPT models, hence an API key is needed.

In [1]:
import os

os.environ['OPENAI_API_KEY'] = ''

### Models & Model Wrappers: The Core of LangChain
In LangChain, models are at the heart of the framework, enabling the integration and manipulation of large language models (LLMs) to fit specific application needs. Here, we explore how to initialize and use models with LangChain wrappers. These wrappers simplify the interaction with different LLMs, providing a unified interface to enhance usability and flexibility. We'll demonstrate using OpenAI's models as an example, illustrating both a standard LLM and Chat model.

In [2]:
from langchain_openai import OpenAI, ChatOpenAI

# Initialize the standard LLM wrapper with the OpenAI API key.
llm = OpenAI()

# Initialize the Chat-oriented LLM wrapper with the OpenAI API key.
chat_llm = ChatOpenAI()

In [3]:
# Example of invoking the standard LLM with a prompt that's intentionally incomplete
response = llm.invoke("ELI5: What does a ")
# Normal LLMs like this simply predict and output the most likely next words following the given prompt.
print(response)

401(k) do?

A 401(k) is a retirement savings plan that is offered by employers. It allows employees to contribute a portion of their salary to a retirement account, which is then invested in a variety of assets such as stocks, bonds, and mutual funds.

The main purpose of a 401(k) is to provide a way for individuals to save for their retirement. The contributions made to the account are typically pre-tax, meaning they are deducted from the employee's paycheck before taxes are taken out. This reduces the amount of income that is subject to taxes, which can help individuals save money in the short-term.

In addition to tax benefits, 401(k) plans often offer employer matching contributions, where the employer will contribute a certain percentage of the employee's salary to their 401(k) account. This can help boost retirement savings even further.

The money in a 401(k) account grows over time through the investments made by the account holder. These investments are managed by a financial 

In [4]:
# ChatBots, unlike standard LLMs, are designed to engage in conversations and provide answers. 

# Here we invoking the chat-oriented LLM with the same incomplete prompt
response = chat_llm.invoke("ELI5: What does a ")

# The `invoke()` function here does additional processing to handle the conversation dynamics, transforming your prompt into a 'HumanMessage' and responding with an 'AI Message'.
# If correctly trained, the chatbot should respond with that it does not understand the question, as it is not fully formed. It does not always do that though xD
response

AIMessage(content='What is "a" in this context? Can you provide more information or context so that I can better explain it to you?', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 15, 'total_tokens': 41}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-4c88b1be-cb2c-4294-b5ec-31959a16621f-0')

## Prompts
A prompt for a language model is a set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [5]:
from langchain_core.prompts import PromptTemplate

# Let's create a simple prompt template. This is just a static string that we can use later to pass to an LLM.
simple_prompt = PromptTemplate.from_template("This will be the prompt for my next call")
simple_prompt

PromptTemplate(input_variables=[], template='This will be the prompt for my next call')

In [6]:
# However, prompts can be dynamic too. Here, we define placeholders within the prompt that can be filled in later.
# This makes it easy to customize the prompt without rewriting the whole thing each time.
joke_prompt = PromptTemplate.from_template("Tell me a {adjective} joke about {content}.")
joke_prompt 

PromptTemplate(input_variables=['adjective', 'content'], template='Tell me a {adjective} joke about {content}.')

You can prompt an LLM simply by invoking the LLM with the prompt.

In [7]:
# Before using our simple prompt with the LLM, we need to format it to convert it from a template to a string.
formatted_simple_prompt = simple_prompt.format()
print("Formatted prompt:", formatted_simple_prompt)  # Showing the formatted prompt.

# Now we'll use this formatted prompt to ask our language model to generate a response.
llm.invoke(formatted_simple_prompt)

Formatted prompt: This will be the prompt for my next call


"\n\nWe will be discussing the importance of self-care and self-love in our daily lives. How do you prioritize taking care of yourself and showing love to yourself? What are some ways you practice self-care and self-love? Share your tips and experiences with the group and let's support each other in our journey towards a healthier and happier self."

In [8]:
# Formatting a prompt with variables. We provide values for the placeholders we defined earlier.
formatted_joke_prompt = joke_prompt.format(adjective='funny', content='chickens')
print("Formatted prompt:", formatted_joke_prompt)  # Take a look at how the prompt looks now.

# Let's see what kind of joke our LLM comes up with using this custom prompt.
llm.invoke(formatted_joke_prompt)

Formatted prompt: Tell me a funny joke about chickens.


'\n\nWhy did the chicken go to the seance?\n\nTo talk to the other side of the road!'

## Chains
Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step.

With [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/expression_language/), we can _chain_ together operations.

In [9]:
# Let's look at how we can create a chain using LangChain. Here, we'll chain a prompt template with a chat-based LLM.
chain = joke_prompt | chat_llm

# Alternatively, you can use the 'pipe' method which explicitly shows the operation's flow from one component to the next.
chain = joke_prompt.pipe(llm)

# The '|' is known as the 'piping' operator, similar to Bash, where the output from the left operation serves as the input to the right operation.
# This chaining effectively turns the process into a single callable sequence.

# Now, let's invoke the chain. We need to provide the values for our prompt template's placeholders.
chain.invoke({'adjective': 'cringy', 'content': 'consultant'})

'\n\nWhy did the consultant cross the road?\nTo get to the other PowerPoint presentation!'

Chains can be as complicated as you want them to be! They can involve processing runnables in parallel, include various tools and functions, and even integrate document retrieval capabilities and much more. This example is a basic introduction to get you started, but feel free to let your imagination and creativity take the lead in designing more complex workflows!

# Retrievers
Retrievers in LangChain are components designed to efficiently retrieve relevant documents or pieces of information from a larger corpus based on the input query. This capability is crucial for applications like question answering systems, where the model needs to access and understand specific information to generate accurate responses. The following example demonstrates how to set up a complete retrieval chain using various LangChain tools, from loading documents to querying them with a language model.

In [10]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

In [11]:
# First, we need some 'additional information' that can be retrieved.
# For this, we will take the LangSmith user guide, which we can load in with beautifulsoup.
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

# Embeddings are used to convert text to a numerical format that a machine can understand.
# Here we use the OpenAIEmbeddings module.
embeddings = OpenAIEmbeddings()

# However, to use the embeddings, we need to split the document up into bite sized pieces that can be embedded.
# For this, we can use a text splitter.
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

# Now we can use the embeddings module and the splitted text to create a vector store.
vector = FAISS.from_documents(documents, embeddings)


# Alright, now we need a prompt that allows for the use of additional context.
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
Question: {input}""")

# Next, we create a chain that integrates document processing with our chat model.
# DOC: The input is a dictionary that must have a "context" key that maps to a List[Document], and any other input variables expected in the prompt. The Runnable return type depends on output_parser used.
document_chain = create_stuff_documents_chain(chat_llm, prompt) # This is not the same as chaining! This us just a function! 

# The retriever is set up to fetch relevant documents based on the query.
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Invoking the retrieval chain with a specific question to see how the system processes and retrieves relevant information.
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])


# To recap what this does:
# We input a question in string format
# FAISS does a similarity search to retrieve documents from the vector store and adds them to the 'context' variable. https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval.create_retrieval_chain.html
# The context is fed to the prompt, which is formatted, and fed to the LLM, which generates a nice output.

LangSmith can help with testing by allowing developers to create datasets, run tests on their LLM applications, upload test cases in bulk, create test cases on the fly, export test cases from application traces, and run custom evaluations to score test results. The platform also provides a comparison view to track and diagnose regressions in test scores across multiple revisions of the application. Additionally, LangSmith offers a playground environment for rapid iteration and experimentation, as well as features for beta testing, capturing feedback, annotating traces, and adding runs to datasets to refine and improve application performance.


## Agents (& Tools, and Parsers)
Agents in LangChain represent a dynamic approach to handling sequences of actions. Unlike chains, where a sequence of actions is explicitly defined in the code, agents utilize a language model as a reasoning engine to dynamically determine the sequence and type of actions to execute based on context. This flexibility allows agents to adapt to changing inputs and complex decision-making scenarios.

In [12]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

# Initialize wikipedia tool, such that the agent has something to do ;) 
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Make sure to bind the tool to the llm, such that the llm can actually use the tool!
llm_with_tools = chat_llm.bind_tools([wikipedia_tool])


# Initialize the prompt.
# For agents, you need to think of how the agent is allowed to 'think'. 
# Here, this is done by giving the agent access to an 'agent_scratchpad', where it can put all intermediate messages in.
wikipedia_topic_link_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "It is your task to make the most random and crazily weird connections between different topics by searching through wikipedia."),
        ("user", "What is the relation between {topic1} and {topic2}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# A helper function to visualize what the agent is actually doing:
def input_print(param):
    print("\nINPUT:", param)
    return param

# Intialize the agent, by chaining together several parts.
agent = (
    {   # This first part just formats inputs
        # This is primarily important for the agent_scratchpad.
        "topic1": lambda x: x['topic1'],
        "topic2": lambda x: x['topic2'],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
        "print": lambda x: input_print(x),
    }
    | wikipedia_topic_link_prompt       # The inputs are fed to the prompt.
    | llm_with_tools                    # The formatted prompt is handed down to the LLM, which might, or might not, invoke tools.
    | OpenAIToolsAgentOutputParser()    # The output of the llm is formatted by a customer output parser which formats the LLM output such that it can be used as input again for new tools.
)

agent_executor = AgentExecutor(agent=agent, tools=[wikipedia_tool], verbose=True)

agent_executor.invoke({"topic1": "singing", "topic2": "skiing"})

# Looking at the output, the following run is happening:
#    Run 1:
#       The agent analyzes the initial prompt, and determines it wants to search for 'singing' on wikipedia, and retrieves the content.
#       Having the content returned, the parser formats it such that it can be used by the agent again.
#    Run 2:
#       The agent analyzes the prompt (now containing the first wikipedia search), and determines it wants to search for 'skiing' on wikipedia, and retrieves the content.
#       Having the content returned, the parser formats it such that it can be used by the agent again.
#    Run 3: 
#       The agent now has both information on skiing and singing, and determines it can form an answer.
#       As such the agent finalizes its output.



> Entering new AgentExecutor chain...

INPUT: {'topic1': 'singing', 'topic2': 'skiing', 'intermediate_steps': []}

Invoking: `wikipedia` with `singing`


Page: Singing
Summary: Singing is the act of creating musical sounds with the voice. A person whose profession is singing is called a singer, artist or vocalist (in jazz and/or popular music). Singers perform music (arias, recitatives, songs, etc.) that can be sung with or without accompaniment by musical instruments. Singing is often done in an ensemble of musicians, such as a choir. Singers may perform as soloists or accompanied by anything from a single instrument (as in art songs or some jazz styles) up to a symphony orchestra or big band. Different singing styles include art music such as opera and Chinese opera, Indian music, Greek music, Japanese music, and religious music styles such as gospel, traditional music styles, world music, jazz, blues, ghazal, and popular music styles such as pop, rock, and electronic dance music.


{'topic1': 'singing',
 'topic2': 'skiing',
 'output': 'The connection between singing and skiing could be through the concept of "bathroom singing" and "alpine skiing." Bathroom singing, also known as singing in the bathroom, is a widespread phenomenon where people enjoy singing due to the acoustics created by hard wall surfaces. Similarly, alpine skiing, or downhill skiing, is the pastime of sliding down snow-covered slopes on skis at ski resorts. Maybe there\'s a quirky correlation between the acoustics of bathroom singing and the thrill of skiing down snowy slopes!'}

### References!

There are some useful links that can help you get further faster. Here is a list:
- [On output parser](https://python.langchain.com/docs/modules/model_io/output_parsers/)
- [On retrievers](https://python.langchain.com/docs/modules/data_connection/retrievers/)
- [On how tools work](https://python.langchain.com/docs/modules/tools/)
- [On what type of tools are available](https://python.langchain.com/docs/integrations/tools/)
- [On Agents](https://python.langchain.com/docs/modules/agents/)
- [On all LLMs which are integrated](https://python.langchain.com/docs/integrations/platforms/)